# Building a simple news recommendation system

The goal of this example is to train a very simple news recommendation system. We will
- prepare the training data in parallel with Ray
- train a simple model that classifies article titles as "popular" or "less popular" using scikit learn and
- find good hyperparameter settings for the model with Tune, Ray's parallel hyperparameter optimization library.

### Downloading and preparing the training data

<html><img src="newsreader_1.png"/></html>

First we will download and uncompress 400,000 hackernews submissions. This is a small subset of the articles that have been submitted to https://news.ycombinator.com. The data includes the title of each submission and its score, which roughly corresponds to the number of upvotes. There are 4 batches of JSON files that contain the information, named `submission-1.json` through `submission-4.json`. The first couple lines of the first file will be printed below by the `head` command.

In [1]:
# !wget -nc https://s3-us-west-2.amazonaws.com/ray-tutorials/hackernews.zip
# !unzip -o hackernews.zip
# !head -n 2 submission-1.json

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import numpy as np
import pandas as pd
import ray
import time

In [3]:
ray.init(num_cpus=4, ignore_reinit_error=True)

2024-04-16 12:02:56,319	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.10.0


The function below parses a chunk of the data and produces a pandas DataFrame with the titles and scores of the submissions.

In [4]:
def parse_hn_submissions(path):
    with open(path, "r") as f:
        records = []
        for line in f.readlines():
            body = json.loads(line)["body"]
            records.append({"data": body["title"], "score": body["score"]})
        return pd.DataFrame(records)

We now process all the data chunks and concatenate them into a single dataframe:

In [5]:
start_time = time.time()

files = ["submission-" + str(i) + ".json" for i in range(1, 5)]
records = [parse_hn_submissions(file) for file in files]
df = pd.concat(records)

end_time = time.time()
duration = end_time - start_time
print("Took {} seconds to parse the hackernews submissions".format(duration))

df.head()

Took 2.5035617351531982 seconds to parse the hackernews submissions


,data,score
0,The PeopleWeb | Mark Pincus Blog (March 2005),3
1,Computer science and programming are two separ...,1
2,Don't Go It Alone: Create an Advisory Board,1
3,Wikileaks Secret Dreams,1
4,MakeMyTrip.com: Is eCommerce in India Finall...,1


**EXERCISE:** Modify the code above to parallelize the parsing of the four files with Ray!

**Note**: In Binder this will not lead to a speedup (in fact, it will be slower) due to constrained resources (each Binder instance is shared with many other people). On an uncontended EC2 instance, we get **4.25s** for the serial code and **1.34s** for the parallel version.

We use the following lines to determine a cutoff of what we consider a "good" article. The median score for articles is 1, so we want to label articles with score higher than that as class "1" and everything else as "0".

In [6]:
df["score"].median()

1.0

In [7]:
df["target"] = df["score"] > 1.0

We are now done preparing the data and can start training a model.

### Training a model

<html><img src="newsreader_2.png"/></html>

First we split the data into a train and test set.

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

The following defines a pipeline that first converts the title of the submission to a bag of words and then applies an SVM for the actual classification. Note that we are fitting a very simple SVM here due to the computational restrictions of Binder. With more resources, a state-of-the-art model like [BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) would be a better choice, in this case the code would be structured similarly.

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("clf", SGDClassifier(loss="hinge", penalty="l2",
                          alpha=0.001,
                          max_iter=5, tol=1e-3,
                          warm_start=True))])
result = pipeline.fit(train.data, train.target)

predicted = result.predict(train.data)
print("Accuracy on the training set is {}".format(np.mean(predicted == train.target)))

/home/zl20/kishu-venv/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Accuracy on the training set is 0.584815625


In [10]:
predicted = pipeline.predict(test.data)
print("Accuracy on the test set is {}".format(np.mean(predicted == test.target)))

Accuracy on the test set is 0.5818375


We can also classify new titles as follows:

In [11]:
pipeline.predict(["Iconic consoles of the IBM System/360 mainframes, 55 years old today",
                  "Are Banned Drugs in Your Meat?"])

array([ True, False])

### Hyperparameter tuning
<html><img src="newsreader_3.png"/></html>

Now let's try to improve these results by doing some hyperparameter tuning. Hyperparameter tuning is the process of finding the best parameters for the learning algorithm. These parameters are typically few numbers like learning rate schedule (i.e. how large steps to take in each iteration), regularization parameters or size of the model. By tuning these knobs, we can typically make the model perform better. Tune supports a number of different algorithms to perform hyperparameter tuning. The simplest is a grid search where we just exhaustively try out different values for the parameters. More sophisticated algorithms include hyperband and population based training. If you want to learn more about these, check out the [tune documentation](https://ray.readthedocs.io/en/latest/tune.html). 

In [12]:
import os
import pickle
from ray import tune

First we need to put the training data into the object store (to make sure it will be re-used between training runs), and define the objective function. The objective function `train_func` takes two arguments: The `config` argument which contains the hyperparameters for that hyperparameter run. The `reporter` object can be used to report the performance of these hyperparameters back to tune so it can select the next trial based on the performance of the past ones.

**EXERCISE**: Inside the `train_func`, instantiate the training pipeline as above and replace the concrete value of $\alpha$ with the value `config["alpha"]` that is passed in by Tune.

The following function instantiates a model corresponding to the hyperparameters in `config`, runs 5 iterations of training and saves the model parameters to a checkpoint file.

In [13]:
train_id = ray.put(train)
test_id = ray.put(test)

def train_func(config):
    pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("clf", SGDClassifier(loss="hinge", penalty="l2",
                          alpha=config["alpha"],
                          max_iter=50, tol=1e-3,
                          warm_start=True))])
    train = ray.get(train_id)
    test = ray.get(test_id)
    for i in range(5):
        # Perform one epoch of SGD
        X = pipeline.named_steps["vect"].fit_transform(train.data)
        pipeline.named_steps["clf"].partial_fit(X, train.target, classes=[0, 1])
        with open("model.pkl", "wb") as f:
            pickle.dump(pipeline, f)
        #reporter(mean_accuracy=np.mean(pipeline.predict(test.data) == test.target))  # report metrics

We can then get the best setting for the regularization parameter $\alpha$ as follows. **You should expect the training to take about 4-5 minutes**.

In [14]:
all_trials = tune.run(
    train_func,
    name="news_recommendation",
    # With the "stop" parameter, you could also specify a stopping criterion.
    config={"alpha": tune.grid_search([1e-3, 1e-4, 1e-5, 1e-6])}
)

2024-04-16 12:03:05,098	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-04-16 12:03:05,104	INFO tensorboardx.py:196 -- pip install "ray[tune]" to see TensorBoard files.
2024-04-16 12:03:05,104	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


2024-04-16 12:03:18,717	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-16 12:03:18,718	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/home/zl20/ray_results/news_recommendation' in 0.0036s.


Trial train_func_30f58_00000 completed. Last result: 
Trial train_func_30f58_00003 completed. Last result: 
Trial train_func_30f58_00002 completed. Last result: 
Trial train_func_30f58_00001 completed. Last result: 


2024-04-16 12:03:18,722	INFO tune.py:1048 -- Total run time: 13.62 seconds (13.61 seconds for the tuning loop).


The best model can now be loaded and evaluated like so: